In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import torch
import torch.nn as nn

import torch.nn.functional as F
import dgl
import dgl.nn as dglnn
import torch.nn as nn
from dgl import AddSelfLoop

from model_2d.NMR_gcn import NMR_GCN

from train_evaluate_2d import NMR_prediction

from preprocess.create_node_embedding.node_embeddings_glycoscience import create_node_embeddings

from preprocess.build_adjaency_matrix.create_adjaency_matrix_glycoscience import build_adjacency_matrix

from preprocess.create_graph.create_graph_data_glycosciencedb import create_graph

# from dgl import save_graphs, load_graphs
# from tqdm import tqdm
from dgl import save_graphs, load_graphs


##### 1, Create Node embeddings

##### Initialize atom-level node embeddings based on extracted atom and monosaccharide features. 

##### Due to the inconsistent of data formating, as GlycoscienceDB is retrieved from various labs, the method of creating node embedding is slighly different from GODESS.

1, atom_dim: dimension of the atom name that indicate ring position, Eg: 04, C4, C3...

2, mono_dim: dimension of the monosaccharide name, glc.

3, ab_dim: dimension of the anomeric orientation of hydroxyl group.

4, dl_dim: dimension of the fischer convention, fischer d, l.

5, pf_dim: dimension of the number of carbon p, f.

6, residual_dim: dimension of the residual number starting from the root as number 1.



In [2]:
atom_dim = 256

residual_dim = 128

mono_dim = 64
ab_dim = 64
dl_dim = 64
pf_dim = 64

seed=9721

In [3]:
C = create_node_embeddings(seed=seed)

df_atom_embedding, df_residual_embedding, df_monosaccharide_embedding, df_ab_embedding, df_dl_embedding, df_pf_embedding =\
C.create_all_embeddings(atom_dim=atom_dim, residual_dim=residual_dim, mono_dim=mono_dim, 
                       ab_dim=ab_dim, dl_dim=dl_dim, pf_dim=pf_dim)



100%|████████████████████████████████████████| 299/299 [00:00<00:00, 678.88it/s]


In [4]:
df_atom_embedding.to_csv(C.out_atom_embed, index=False)
df_residual_embedding.to_csv(C.out_residual_embed, index=False)
df_monosaccharide_embedding.to_csv(C.out_monosaccharide_embed, index=False)

df_ab_embedding.to_csv(C.out_bound_ab, index = False) 
df_dl_embedding.to_csv(C.out_bound_dl, index = False) 
df_pf_embedding.to_csv(C.out_carbon_pf, index = False) 

##### 2, Create adjaency matrix

##### Build adjaency matrix based on threshold distance between atoms

1, threshold_carbon: we construct edges between two carbon atom if their distance is within 1.65Å. 

2, threshold_hydrogen: we construct edges between two hydrogen atom if their distance is within 1.18Å. 

3, threshold_general: for all other atoms, we construct edges between distance two atom is within 1.5Å. 

4, threshold_interaction: we construct interaction edges between atoms if their distance is with in 5Å. 


In [5]:
B = build_adjacency_matrix()

In [6]:
B.calculate_all_matrix()

100%|█████████████████████████████████████████| 299/299 [00:04<00:00, 64.92it/s]


##### 3, Create glycan graph

##### create glycan graph from prebuilt node embeddings and adjaency matrix, every glycan is considered as a graph and every atom is consided as a node

1, num_test: number of testing glycans, we randomly select 20% of the glycans as test set


In [7]:
num_test = 60 # 299 * 20% 

In [8]:
Create = create_graph(num_test=num_test, seed=seed, use_mono=0, use_ab=0, use_dl=0, use_pf=0)
g, test_index = Create.create_all_graph()

--------------------------loading NMR Graph-------------------------------


100%|█████████████████████████████████████████| 299/299 [00:04<00:00, 65.54it/s]


In [9]:
g

Graph(num_nodes=27232, num_edges=84308,
      ndata_schemes={'feat': Scheme(shape=(256,), dtype=torch.float32), 'shift_value': Scheme(shape=(), dtype=torch.float32), 'Carbon_Hydrogen_mask': Scheme(shape=(), dtype=torch.bool), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'train_carbon_mask': Scheme(shape=(), dtype=torch.bool), 'test_carbon_mask': Scheme(shape=(), dtype=torch.bool), 'train_hydrogen_mask': Scheme(shape=(), dtype=torch.bool), 'test_hydrogen_mask': Scheme(shape=(), dtype=torch.bool), 'glycan_index': Scheme(shape=(), dtype=torch.int64), 'atom_index_list': Scheme(shape=(), dtype=torch.int8), 'feat1_atom': Scheme(shape=(256,), dtype=torch.float32), 'feat2_mono': Scheme(shape=(0,), dtype=torch.float32), 'feat3_ab': Scheme(shape=(0,), dtype=torch.float32), 'feat4_dl': Scheme(shape=(0,), dtype=torch.float32), 'feat5_pf': Scheme(shape=(0,), dtype=torch.float32)}
      edata_schemes={})

##### 4, Train a 4-layer GNN model

In [9]:
num_epoch = 1000
lr = 1e-2


In [10]:
# in_size = atom_dim + mono_dim + ab_dim + dl_dim + pf_dim
in_size = atom_dim

hidden_size_1 = int(in_size / 2)
hidden_size_2 = 256
hidden_size_3 = 128
hidden_size_4 = 32

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

g = g.to(device)
features = g.ndata["feat"]
labels = g.ndata["shift_value"]
# masks = g.ndata['train_mask'], g.ndata['test_mask']


# masks = g.ndata['train_hydrogen_mask'], g.ndata['test_hydrogen_mask']
masks = g.ndata['train_carbon_mask'], g.ndata['test_carbon_mask']
# masks = g.ndata['train_mask'], g.ndata['test_mask']

print(features.dtype)
print(labels.dtype)
# model = NMR_GCN(in_size=576, hid_size=[256, 128, 64, 32], out_size=1).to(device)
model = NMR_GCN(in_size=in_size, hid_size=[hidden_size_1, hidden_size_2,
                                       hidden_size_3, hidden_size_4], out_size=1).to(device)
# model training

# NMR_prediction = NMR_prediction(results_dir='experimental_data_combined/graph_combined/experimental_results_all_hydrogen.csv',
#                                 model_dir='experimental_data_combined/graph_combined/Model_hydrogen_experiment_training.pt',
#                                num_epoch = num_epoch, 
#                                lr = lr)


NMR_prediction = NMR_prediction(results_dir='glycosciencedb/results/training_carbon.csv',
                                results_dir_test = 'glycosciencedb/results/testing_carbon.csv',
                                model_dir='glycosciencedb/results/Model_Godess_carbon.pt', 
                               num_epoch = num_epoch, 
                               lr = lr)


print("Training...")
NMR_prediction.train(g, features, labels, masks, model)

# test the model
print("Testing...")
saved_model = NMR_GCN(in_size=in_size, hid_size=[hidden_size_1, hidden_size_2,
                                                 hidden_size_3, hidden_size_4], out_size=1).to(device)
saved_model.load_state_dict(torch.load(NMR_prediction.model_dir))

# acc = NMR_prediction.evaluate(g, features, labels, masks[0], saved_model, print_out=True)
acc1 = NMR_prediction.evaluate(g, features, labels, masks[0], saved_model, save_train=True, save_test = False)

acc2 = NMR_prediction.evaluate(g, features, labels, masks[1], saved_model, save_train=False, save_test = True)


print("train RMSE {:.4f}".format(acc1))
print("test RMSE {:.4f}".format(acc2))


# acc1 = NMR_prediction.evaluate(g, features, labels, masksc[0], saved_model, save_train=True, save_test = False)

# acc2 = NMR_prediction.evaluate(g, features, labels, masksc[1], saved_model, save_train=False, save_test = True)


# print("train RMSE {:.4f}".format(acc1))
# print("test RMSE {:.4f}".format(acc2))

torch.float32
torch.float32
Training...


  1%|▌                                        | 13/1000 [00:00<00:45, 21.60it/s]

1247
5263
Epoch 00000 | Loss 6110.9092 | train_RMSE 74.9357 | test_RMSE 76.2381 
1247
5263
Epoch 00001 | Loss 5616.6001 | train_RMSE 51.9673 | test_RMSE 52.4975 
1247
5263
Epoch 00002 | Loss 2701.2690 | train_RMSE 55.9235 | test_RMSE 55.8846 
1247
5263
Epoch 00003 | Loss 3163.4854 | train_RMSE 26.6349 | test_RMSE 25.8076 
1247
5263
Epoch 00004 | Loss 718.3294 | train_RMSE 41.9975 | test_RMSE 42.3628 
1247
5263
Epoch 00005 | Loss 1764.9888 | train_RMSE 42.4794 | test_RMSE 42.9797 
1247
5263
Epoch 00006 | Loss 1808.1899 | train_RMSE 32.5931 | test_RMSE 32.7747 
1247
5263
Epoch 00007 | Loss 1070.5121 | train_RMSE 20.3061 | test_RMSE 19.4908 
1247
5263
Epoch 00008 | Loss 424.8855 | train_RMSE 33.8214 | test_RMSE 33.3100 
1247
5263
Epoch 00009 | Loss 1167.4573 | train_RMSE 24.7566 | test_RMSE 24.1546 
1247
5263
Epoch 00010 | Loss 635.8628 | train_RMSE 15.0404 | test_RMSE 14.8700 
1247
5263
Epoch 00011 | Loss 241.5653 | train_RMSE 22.2777 | test_RMSE 22.9447 
1247
5263
Epoch 00012 | Loss 507

  4%|█▌                                       | 37/1000 [00:00<00:16, 57.66it/s]

5263
Epoch 00023 | Loss 138.3204 | train_RMSE 11.0900 | test_RMSE 10.8891 
1247
5263
Epoch 00024 | Loss 142.8125 | train_RMSE 11.4947 | test_RMSE 11.4026 
1247
5263
Epoch 00025 | Loss 155.2944 | train_RMSE 9.9084 | test_RMSE 10.2449 
1247
5263
Epoch 00026 | Loss 119.3613 | train_RMSE 9.7771 | test_RMSE 10.6227 
1247
5263
Epoch 00027 | Loss 113.6879 | train_RMSE 11.4326 | test_RMSE 12.4578 
1247
5263
Epoch 00028 | Loss 145.3002 | train_RMSE 11.7624 | test_RMSE 12.7888 
1247
5263
Epoch 00029 | Loss 156.8831 | train_RMSE 10.2784 | test_RMSE 11.1957 
1247
5263
Epoch 00030 | Loss 122.9862 | train_RMSE 9.0755 | test_RMSE 9.6936 
1247
5263
Epoch 00031 | Loss 105.2091 | train_RMSE 9.4236 | test_RMSE 9.6865 
1247
5263
Epoch 00032 | Loss 108.9689 | train_RMSE 9.1313 | test_RMSE 9.2582 
1247
5263
Epoch 00033 | Loss 105.5939 | train_RMSE 8.0644 | test_RMSE 8.3271 
1247
5263
Epoch 00034 | Loss 85.4746 | train_RMSE 8.4044 | test_RMSE 8.9294 
1247
5263
Epoch 00035 | Loss 91.9651 | train_RMSE 9.3610 |

...

1247
5263
Epoch 00937 | Loss 26.5092 | train_RMSE 3.9511 | test_RMSE 4.3181 
1247
5263
Epoch 00938 | Loss 26.4122 | train_RMSE 3.9517 | test_RMSE 4.3403 
1247
5263
Epoch 00939 | Loss 26.5512 | train_RMSE 3.9517 | test_RMSE 4.3470 
1247
5263
Epoch 00940 | Loss 27.2389 | train_RMSE 4.0861 | test_RMSE 4.5488 
1247
5263
Epoch 00941 | Loss 26.5187 | train_RMSE 4.1419 | test_RMSE 4.5422 
1247
5263
Epoch 00942 | Loss 27.6935 | train_RMSE 4.5144 | test_RMSE 5.0046 
1247
5263
Epoch 00943 | Loss 30.8484 | train_RMSE 4.6467 | test_RMSE 4.9343 
1247
5263
Epoch 00944 | Loss 33.0594 | train_RMSE 5.6004 | test_RMSE 6.0069 
1247
5263
Epoch 00945 | Loss 40.0094 | train_RMSE 6.0212 | test_RMSE 6.2593 
1247
5263
Epoch 00946 | Loss 47.1703 | train_RMSE 7.3295 | test_RMSE 7.7556 
1247
5263
Epoch 00947 | Loss 64.2129 | train_RMSE 6.9424 | test_RMSE 7.1702 
1247
5263
Epoch 00948 | Loss 58.9680 | train_RMSE 7.6150 | test_RMSE 8.0721 
1247
5263
Epoch 00949 | Loss 67.7126 | train_RMSE 5.9941 | test_RMSE 6.1813 

 98%|██████████████████████████████████████▍| 985/1000 [00:08<00:00, 123.27it/s]

1247
5263
Epoch 00962 | Loss 26.2592 | train_RMSE 4.0186 | test_RMSE 4.4296 
1247
5263
Epoch 00963 | Loss 26.8488 | train_RMSE 4.1394 | test_RMSE 4.5824 
1247
5263
Epoch 00964 | Loss 26.4125 | train_RMSE 4.0378 | test_RMSE 4.4250 
1247
5263
Epoch 00965 | Loss 27.3683 | train_RMSE 4.1329 | test_RMSE 4.5729 
1247
5263
Epoch 00966 | Loss 27.0532 | train_RMSE 4.0082 | test_RMSE 4.4179 
1247
5263
Epoch 00967 | Loss 26.1409 | train_RMSE 3.9888 | test_RMSE 4.4226 
1247
5263
Epoch 00968 | Loss 26.6089 | train_RMSE 3.9997 | test_RMSE 4.4068 
1247
5263
Epoch 00969 | Loss 27.4408 | train_RMSE 3.9852 | test_RMSE 4.3632 
1247
5263
Epoch 00970 | Loss 25.7320 | train_RMSE 3.9983 | test_RMSE 4.3877 
1247
5263
Epoch 00971 | Loss 27.4966 | train_RMSE 3.9781 | test_RMSE 4.3864 
1247
5263
Epoch 00972 | Loss 25.5688 | train_RMSE 3.9842 | test_RMSE 4.4129 
1247
5263
Epoch 00973 | Loss 25.9044 | train_RMSE 4.0049 | test_RMSE 4.4322 
1247
5263
Epoch 00974 | Loss 25.6217 | train_RMSE 3.9587 | test_RMSE 4.3552 

100%|██████████████████████████████████████| 1000/1000 [00:08<00:00, 111.66it/s]


1247
5263
Epoch 00987 | Loss 25.6608 | train_RMSE 3.9550 | test_RMSE 4.3704 
1247
5263
Epoch 00988 | Loss 24.8434 | train_RMSE 3.9514 | test_RMSE 4.3509 
1247
5263
Epoch 00989 | Loss 26.3691 | train_RMSE 4.0402 | test_RMSE 4.4588 
1247
5263
Epoch 00990 | Loss 26.0241 | train_RMSE 3.9780 | test_RMSE 4.3409 
1247
5263
Epoch 00991 | Loss 26.7453 | train_RMSE 4.0155 | test_RMSE 4.4091 
1247
5263
Epoch 00992 | Loss 25.8460 | train_RMSE 3.9209 | test_RMSE 4.3078 
1247
5263
Epoch 00993 | Loss 25.5577 | train_RMSE 3.9388 | test_RMSE 4.3373 
1247
5263
Epoch 00994 | Loss 25.7256 | train_RMSE 4.2857 | test_RMSE 4.7494 
1247
5263
Epoch 00995 | Loss 28.6466 | train_RMSE 4.4476 | test_RMSE 4.7660 
1247
5263
Epoch 00996 | Loss 30.2405 | train_RMSE 4.6832 | test_RMSE 5.1634 
1247
5263
Epoch 00997 | Loss 32.9022 | train_RMSE 4.5729 | test_RMSE 4.8610 
1247
5263
Epoch 00998 | Loss 32.9474 | train_RMSE 4.7261 | test_RMSE 5.1852 
1247
5263
Epoch 00999 | Loss 32.6423 | train_RMSE 4.4942 | test_RMSE 4.7882 